In [2]:
import pandas as pd
import numpy as np
from numpy import linalg as LA
from sklearn.preprocessing import StandardScaler
import os

In [3]:
# format pandas output

pd.options.display.float_format = '{:.2e}'.format

In [4]:
data = pd.read_parquet("data/clean_full_bbo_data.parquet")
data.head(2)

,time,Stock,bid_vwa,ask_vwa,vwap_mid_price
0,2008-04-23 14:26,PG.N,6.71e+01,6.71e+01,6.71e+01
1,2008-08-07 13:28,C.N,1.89e+01,1.89e+01,1.89e+01


In [5]:
data['time'] = pd.to_datetime(data['time'])

# Sort the DataFrame
data = data.sort_values(by=["time", "Stock"])

# Create the pivot table
pivot_table = data.pivot(index="time", columns="Stock", values="vwap_mid_price")

pivot_table = pivot_table.sort_index(axis=1)

# Forward & Backward fill
pivot_table = pivot_table.fillna(method='bfill', axis=0)
pivot_table = pivot_table.fillna(method='ffill', axis=0)

/var/folders/cg/lm106ggx1z59db7x0_wgfghm0000gn/T/ipykernel_21747/3679780221.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pivot_table = pivot_table.fillna(method='bfill', axis=0)
/var/folders/cg/lm106ggx1z59db7x0_wgfghm0000gn/T/ipykernel_21747/3679780221.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pivot_table = pivot_table.fillna(method='ffill', axis=0)


In [6]:
# Calculate returns
returns = pivot_table.pct_change() # simple linear returns
log_rets = np.log(1+returns)
log_rets = log_rets.fillna(0)

In [7]:
log_rets

Stock,C.N,EMC.N,HPQ.N,MDT.N,MO.N,NKE.N,PG.N,RTN.N,SO.N,TWX.N,WFC.N
time,,,,,,,,,,,
2008-01-02 09:30:00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
2008-01-02 09:31:00,9.12e-04,-9.63e-04,1.76e-03,1.33e-04,-4.60e-04,0.00e+00,0.00e+00,0.00e+00,-3.61e-03,8.11e-04,-9.38e-04
2008-01-02 09:32:00,3.02e-03,9.37e-04,2.08e-04,2.86e-03,2.82e-03,-8.76e-04,-3.09e-03,1.10e-03,9.64e-04,1.27e-03,-6.11e-04
2008-01-02 09:33:00,-3.20e-03,3.83e-03,1.26e-04,2.19e-04,-2.21e-03,-4.35e-04,-2.16e-03,1.35e-03,1.74e-03,4.58e-04,-1.48e-03
2008-01-02 09:34:00,-3.45e-03,-1.28e-03,-3.53e-05,9.95e-04,-1.03e-03,-2.42e-03,-1.42e-03,-1.55e-03,1.00e-04,8.83e-05,7.96e-04
...,...,...,...,...,...,...,...,...,...,...,...
2008-12-31 15:56:00,1.59e-03,8.25e-04,1.58e-03,2.36e-03,7.13e-04,2.66e-03,7.91e-04,1.69e-03,1.35e-03,1.97e-03,2.25e-03
2008-12-31 15:57:00,1.97e-03,-5.82e-05,5.70e-04,5.58e-04,6.10e-04,-1.19e-03,-3.42e-05,2.05e-04,-2.03e-04,1.97e-03,1.47e-03
2008-12-31 15:58:00,7.97e-04,8.02e-04,-2.69e-04,-9.59e-05,6.90e-04,2.13e-04,4.26e-04,-5.08e-04,8.11e-04,1.39e-03,1.47e-03


In [8]:
# standardize the data
"""
log_rets_standardized = StandardScaler(with_mean=True, with_std=False).fit_transform(log_rets.values)
log_rets = pd.DataFrame(log_rets_standardized, index=log_rets.index, columns=log_rets.columns)

log_rets
"""

'\nlog_rets_standardized = StandardScaler(with_mean=True, with_std=False).fit_transform(log_rets.values)\nlog_rets = pd.DataFrame(log_rets_standardized, index=log_rets.index, columns=log_rets.columns)\n\nlog_rets\n'

In [9]:
def eigenvalue_clipping(lambdas, v, lambda_plus):
    N=len(lambdas)
    
    
    # _s stands for _structure below
    sum_lambdas_gt_lambda_plus = np.sum(lambdas[lambdas>lambda_plus])
    
    sel_bulk = lambdas<=lambda_plus                     
    N_bulk=np.sum(sel_bulk)
    sum_lambda_bulk=np.sum(lambdas[sel_bulk])        
    delta=sum_lambda_bulk/N_bulk                      
    
    lambdas_clean=lambdas
    lambdas_clean[lambdas_clean<=lambda_plus]=delta
    
    
    C_clean=np.zeros((N, N))
    v_m=np.matrix(v)
    
    for i in range(N-1):
        C_clean=C_clean+lambdas_clean[i] * np.dot(v_m[i,].T,v_m[i,]) 
        
    np.fill_diagonal(C_clean,1)
            
    return C_clean    
    


In [10]:
# in sample data

T_in = 60                       # lenght of the rolling window in minutes
start_period_in_sample = 8151   # 1st Febraury 2008
end_period_in_sample = 32417    # last day of April 2008
dT = 5
t0s = np.arange(start_period_in_sample, end_period_in_sample ,dT)    # 16031 is the index st the last date is 29/02/2008
t0s

array([ 8151,  8156,  8161, ..., 32406, 32411, 32416], shape=(4854,))

In [11]:
def compute_weights_GVM(covariance_matrix):
    """
    Function to compute the Global Minimum Variance (GMV) portfolio weights.

    Args:
        Sigma (): Covariance matrix of the asset returns.

    Returns:
        _type_: optimal weights
    """
    covariance_matrix_inv = LA.inv(covariance_matrix)
    weights = covariance_matrix_inv.sum(axis=1) / covariance_matrix_inv.sum()
    return weights

In [12]:
"""
a = log_rets.iloc[20:30]
a = StandardScaler(with_mean=True, with_std=False).fit_transform(a)
a[0]
"""

'\na = log_rets.iloc[20:30]\na = StandardScaler(with_mean=True, with_std=False).fit_transform(a)\na[0]\n'

In [13]:
riks_in_sample_clipped = []
weights_clipped = []
means_in_sample_clipped = []


for t0 in t0s:
  
  t1 = t0 + T_in
  log_rets_cut = log_rets.iloc[t0:t1]
  
  # Compute means, std and correlation
  mean_values = log_rets_cut.mean()
  correlation_matrix = log_rets_cut.corr()
  std_matrix = log_rets_cut.std()
  
  # variables for the eigenvalue clipping
  T, N = log_rets_cut.shape
  q = N / T
  lambda_plus = (1+np.sqrt(q))**2
  eigenvalues, eigenvectors = LA.eig(correlation_matrix)
  
  # Clean Covariance Matrix
  correlation_matrix_clipped = eigenvalue_clipping(eigenvalues, eigenvectors, lambda_plus)
  # correlation_matrix_clipped = pd.DataFrame(correlation_matrix_clipped, index=log_rets_cut.columns, columns=log_rets_cut.columns)
  
  cov_matrix = np.diag(std_matrix) @ correlation_matrix_clipped @ np.diag(std_matrix)
  covariance_matrix_clipped = pd.DataFrame(cov_matrix, index=log_rets_cut.columns, columns=log_rets_cut.columns)

  # print(covariance_matrix_clipped)
  
  # Compute the GMV weights
  weights_GVM_clipped = compute_weights_GVM(covariance_matrix_clipped)
  
  # compute risk in sample
  risk_clipped = np.dot(weights_GVM_clipped.T, np.dot(covariance_matrix_clipped, weights_GVM_clipped))
  
  # Append results
  means_in_sample_clipped.append(mean_values)
  riks_in_sample_clipped.append(risk_clipped)
  weights_clipped.append(weights_GVM_clipped)
  

In [14]:
"""
riks_in_sample_clipped = []
weights_clipped = []
means_in_sample_clipped = []


count = 0
for t0 in t0s:
  
  t1 = t0 + T_in
  log_rets_cut = log_rets.iloc[t0:t1]
  
  # center
  log_rets_cut_centered = StandardScaler(with_mean=True, with_std=False).fit_transform(log_rets_cut)
  
  
  # Compute means, std and correlation
  mean_values = log_rets_cut_centered.mean()
  correlation_matrix = np.corrcoef(log_rets_cut_centered, rowvar=False)
  
  # variables for the eigenvalue clipping
  T, N = log_rets_cut_centered.shape
  q = N / T
  lambda_plus = (1+np.sqrt(q))**2
  eigenvalues, eigenvectors = LA.eig(correlation_matrix)
  
  # Clean Covariance Matrix
  correlation_matrix_clipped = eigenvalue_clipping(eigenvalues, eigenvectors, lambda_plus)
  # correlation_matrix_clipped = pd.DataFrame(correlation_matrix_clipped, index=log_rets_cut.columns, columns=log_rets_cut.columns)
  
  # inverse of corr
  inv_correlation = LA.inv(correlation_matrix_clipped)
  
  
  
  weights_GVM_clipped = inv_correlation @ np.ones(len(inv_correlation)) / (np.ones(len(inv_correlation)) @ inv_correlation @ np.ones(len(inv_correlation)))
  
  # compute risk in sample
  risk_clipped = weights_GVM_clipped @ correlation_matrix_clipped @ weights_GVM_clipped.T
  # risk_clipped = np.dot(weights_GVM_clipped, np.dot(covariance_matrix_clipped, weights_GVM_clipped.T))
  
  # Append results
  means_in_sample_clipped.append(mean_values)
  riks_in_sample_clipped.append(risk_clipped)
  weights_clipped.append(weights_GVM_clipped)
  
"""


'\nriks_in_sample_clipped = []\nweights_clipped = []\nmeans_in_sample_clipped = []\n\n\ncount = 0\nfor t0 in t0s:\n  \n  t1 = t0 + T_in\n  log_rets_cut = log_rets.iloc[t0:t1]\n  \n  # center\n  log_rets_cut_centered = StandardScaler(with_mean=True, with_std=False).fit_transform(log_rets_cut)\n  \n  \n  # Compute means, std and correlation\n  mean_values = log_rets_cut_centered.mean()\n  correlation_matrix = np.corrcoef(log_rets_cut_centered, rowvar=False)\n  \n  # variables for the eigenvalue clipping\n  T, N = log_rets_cut_centered.shape\n  q = N / T\n  lambda_plus = (1+np.sqrt(q))**2\n  eigenvalues, eigenvectors = LA.eig(correlation_matrix)\n  \n  # Clean Covariance Matrix\n  correlation_matrix_clipped = eigenvalue_clipping(eigenvalues, eigenvectors, lambda_plus)\n  # correlation_matrix_clipped = pd.DataFrame(correlation_matrix_clipped, index=log_rets_cut.columns, columns=log_rets_cut.columns)\n  \n  # inverse of corr\n  inv_correlation = LA.inv(correlation_matrix_clipped)\n  \n  \n 

In [15]:
# Create DataFrames

moving_avg_in_sample = pd.DataFrame(means_in_sample_clipped, index=log_rets.index[t0s+T_in], columns=pivot_table.columns)
risks_df_in_sample = pd.DataFrame(riks_in_sample_clipped, index=log_rets.index[t0s+T_in], columns=['Risk'])
weights_df = pd.DataFrame(weights_clipped, index=log_rets.index[t0s+T_in], columns=pivot_table.columns)


In [16]:
weights_df.head()


Stock,C.N,EMC.N,HPQ.N,MDT.N,MO.N,NKE.N,PG.N,RTN.N,SO.N,TWX.N,WFC.N
time,,,,,,,,,,,
2008-02-01 09:30:00,2.64e-01,4.78e-02,4.61e-02,1.32e-01,2.09e-01,1.20e-01,1.32e-01,9.00e-02,-1.79e-01,9.79e-02,3.99e-02
2008-02-01 09:35:00,-3.42e-02,7.20e-02,9.28e-02,9.62e-02,4.58e-01,2.99e-02,-9.01e-02,3.34e-01,3.95e-02,7.39e-03,-5.01e-03
2008-02-01 09:40:00,1.15e-01,8.12e-02,1.46e-02,1.06e-01,1.92e-01,9.15e-04,1.10e-01,2.01e-01,1.45e-01,3.17e-02,2.57e-03
2008-02-01 09:45:00,-3.31e-01,1.88e-01,2.22e-01,9.23e-01,1.42e-01,-6.66e-01,2.78e-01,7.29e-02,4.19e-02,1.31e-01,-2.33e-03
2008-02-01 09:50:00,4.25e-01,-3.22e-01,-6.58e-01,4.65e-01,-9.43e-01,1.99e+00,-7.07e-03,2.57e-01,2.04e-01,-3.55e-01,-5.59e-02


In [17]:
risks_df_in_sample.head()

,Risk
time,
2008-02-01 09:30:00,1.10e-07
2008-02-01 09:35:00,1.13e-07
2008-02-01 09:40:00,1.48e-07
2008-02-01 09:45:00,3.36e-07
2008-02-01 09:50:00,-3.20e-07


## Out-of-Sample

In [18]:
T_in = 60  # lenght of the rolling window in minutes
start_period_out_sample = end_period_in_sample
end_period_out_sample = start_period_out_sample + (end_period_in_sample - start_period_in_sample) # keep same length as in-sample
t0s_out_sample = np.arange(start_period_out_sample, end_period_out_sample ,dT)    # 16031 is the index st the last date is 29/02/2008
t0s_out_sample

array([32417, 32422, 32427, ..., 56672, 56677, 56682], shape=(4854,))

In [19]:
# weights_clipped[0]

In [20]:
riks_out_sample_clipped = []
moving_average_out_sample_clipped = []

for t1 in enumerate(t0s_out_sample):
  
  t2 = t1[1] + T_in
  log_rets_cut = log_rets.iloc[t1[1]:t2]
  
  # Compute means & covariance
  mean_values = log_rets_cut.mean()
  covariance_matrix = log_rets_cut.cov()
  
  # compute risk in sample
  risk = np.dot(weights_clipped[t1[0]].T, np.dot(covariance_matrix, weights_clipped[t1[0]]))
  
  # Append the results to the lists
  moving_average_out_sample_clipped.append(mean_values)
  riks_out_sample_clipped.append(risk)
  

In [21]:
moving_average_out_sample_clipped

[Stock
 C.N      1.12e-04
 EMC.N    5.92e-06
 HPQ.N    1.70e-04
 MDT.N    6.56e-05
 MO.N     8.07e-05
 NKE.N   -2.50e-05
 PG.N    -9.03e-05
 RTN.N    9.17e-06
 SO.N    -2.31e-04
 TWX.N    6.51e-05
 WFC.N    1.35e-04
 dtype: float64,
 Stock
 C.N      1.45e-04
 EMC.N    5.24e-05
 HPQ.N    2.96e-04
 MDT.N    7.99e-05
 MO.N     1.02e-04
 NKE.N    2.74e-05
 PG.N    -5.24e-05
 RTN.N    2.19e-05
 SO.N    -1.71e-04
 TWX.N    1.10e-04
 WFC.N    1.58e-04
 dtype: float64,
 Stock
 C.N      1.40e-04
 EMC.N    4.27e-05
 HPQ.N    3.04e-04
 MDT.N    5.78e-05
 MO.N     1.20e-04
 NKE.N    3.44e-05
 PG.N    -7.17e-05
 RTN.N    2.00e-05
 SO.N    -1.38e-04
 TWX.N    1.41e-04
 WFC.N    2.04e-04
 dtype: float64,
 Stock
 C.N      1.26e-04
 EMC.N    1.77e-05
 HPQ.N    2.80e-04
 MDT.N    6.88e-05
 MO.N     1.16e-04
 NKE.N    1.69e-05
 PG.N    -1.11e-04
 RTN.N    1.66e-05
 SO.N    -1.53e-04
 TWX.N    1.33e-04
 WFC.N    2.06e-04
 dtype: float64,
 Stock
 C.N      1.26e-04
 EMC.N    1.03e-05
 HPQ.N    2.38e-04
 MDT

In [22]:
# Create DataFrames

moving_avg_out_sample_clipped = pd.DataFrame(moving_average_out_sample_clipped, columns=pivot_table.columns)
risks_df_out_sample_clipped = pd.DataFrame(riks_out_sample_clipped, columns=['Risk'])


In [23]:
risks_df_out_sample_clipped.head()

,Risk
0,4.86e-07
1,4.94e-07
2,1.95e-07
3,2.23e-06
4,1.21e-05


In [24]:
risks_df_in_sample.head(3)

,Risk
time,
2008-02-01 09:30:00,1.10e-07
2008-02-01 09:35:00,1.13e-07
2008-02-01 09:40:00,1.48e-07


### Export into CSV

In [25]:
folder_name = "data/correlation_data"

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created.")
else:
    print(f"Folder '{folder_name}' already exists.")


Folder 'data/correlation_data' already exists.


In [26]:
risks_df_in_sample.to_csv(f'{folder_name}/risks_in_sample.csv')
risks_df_out_sample_clipped.to_csv(f'{folder_name}/risk_out_sample.csv')
weights_df.to_csv(f'{folder_name}/weights.csv')
moving_avg_in_sample.to_csv(f'{folder_name}/moving_avg_in_sample.csv')
moving_avg_out_sample_clipped.to_csv(f'{folder_name}/moving_avg_out_sample.csv')